1. Step
- load the dataset as pandas dataframe and extract the edges between nodes and their capacity over all the lanes in one direction together

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import random

df = pd.read_csv("Analysis/GoldCoast.csv")
df_edges = df[["From","To", "Overall_capacity"]]
df_edges

,From,To,Overall_capacity
0,1,1371,1800.0
1,2,2012,3200.0
2,3,2402,2200.0
3,4,1875,800.0
4,5,1880,800.0
...,...,...,...
11135,4806,1495,600.0
11136,4806,3606,600.0
11137,4806,415,9000.0
11138,4807,1433,800.0


2. Step
- load the dataframe as edgelist

In [2]:
G = nx.from_pandas_edgelist(
    df,
    source="From",
    target="To",
    edge_attr=["Overall_capacity"],
    create_using=nx.DiGraph()
)
Nodes = G.number_of_nodes()
Edges = G.number_of_edges()
Nodes, Edges

(4783, 11140)

3. Step
- load the dataframe from a different file that's containing coordinates of the nodes

In [3]:
cor = pd.read_csv('nodes.csv')

4. Step
- extracting candidate pairs of nodes (possible future edges) that are located maximum 2 steps away 
- creating a balanced labeled data set of existing edges and candidate pair (no edges)
- feature extraction 
- creating a dataframe of the labeled dataset of node pairs and their features
- training a random forest classifier on a masked portion of edges ensuring integrity


In [6]:
import random
import numpy as np
import pandas as pd
from collections import deque
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# ------------------- MASK EDGES -------------------
def mask_edges(G, mask_frac=0.1, seed=42):
    """Randomly remove a fraction of edges to use as a test set"""
    random.seed(seed)
    edges = list(G.edges())
    n_mask = int(len(edges) * mask_frac)
    masked_edges = random.sample(edges, n_mask)
    G_train = G.copy()
    G_train.remove_edges_from(masked_edges)
    return G_train, masked_edges

# ------------------- REACHABLE NODES CACHE -------------------
_REACHABLE_CACHE = {}

def compute_reachable_nodes(G, max_distance=2, cache_key=None):
    """Compute reachable nodes once and cache results"""
    if cache_key and cache_key in _REACHABLE_CACHE:
        return _REACHABLE_CACHE[cache_key]
    
    reachable = {u: set() for u in G.nodes()}
    for u in G.nodes():
        visited = {u}
        frontier = deque([(u, 0)])
        while frontier:
            current, dist = frontier.popleft()
            if dist == max_distance: 
                continue
            for nxt in G.successors(current):
                if nxt not in visited:
                    visited.add(nxt)
                    frontier.append((nxt, dist + 1))
                    if nxt != u:
                        reachable[u].add(nxt)
    
    if cache_key:
        _REACHABLE_CACHE[cache_key] = reachable
    
    return reachable

# ------------------- HARD NEGATIVE DATASET -------------------
def create_labeled_dataset_with_hard_negatives(G, cor, positives, max_distance=2, 
                                               negatives_per_positive=1, max_euclid_dist=0.005,
                                               reachable_dict=None):
    """
    G: training graph
    positives: list of (u,v) edges to include
    reachable_dict: OPTIONAL precomputed reachable nodes (saves computation)
    Returns: labeled dataset with realistic negatives
    """
    positions = {row['node']: (row['x'], row['y']) for _, row in cor.iterrows()}
    dataset = []

    # Use precomputed reachable dict or compute new one
    if reachable_dict is not None:
        reachable = reachable_dict
    else:
        reachable = compute_reachable_nodes(G, max_distance)

    # Precompute in-degrees and existing edges for O(1) lookups
    in_degrees = {node: G.in_degree(node) for node in G.nodes()}
    existing_edges = set(G.edges())
    
    for u, v_pos in positives:
        dataset.append((u, v_pos, 1))
        
        if negatives_per_positive > 0:
            candidates = []
            x_u, y_u = positions[u]
            target_degree = in_degrees[v_pos]
            
            # Filter candidates efficiently
            for v in reachable[u]:
                if (u, v) not in existing_edges:
                    x_v, y_v = positions[v]
                    euclid_dist = np.sqrt((x_u - x_v)**2 + (y_u - y_v)**2)
                    if euclid_dist <= max_euclid_dist:
                        degree_diff = abs(in_degrees[v] - target_degree)
                        candidates.append((v, degree_diff))
            
            if candidates:
                candidates.sort(key=lambda x: x[1])
                chosen = [v for v, _ in candidates[:negatives_per_positive]]
                dataset.extend([(u, v_neg, 0) for v_neg in chosen])

    random.shuffle(dataset)
    return dataset

# ------------------- OPTIMIZED FEATURE EXTRACTION -------------------
class FeatureExtractor:
    """Optimized feature extraction with caching"""
    def __init__(self, G, cor):
        self.G = G
        self.cor = cor
        self.positions = {row['node']: (row['x'], row['y']) for _, row in cor.iterrows()}
        self._cache = {}
        
    def extract_features_batch(self, node_pairs):
        """Extract features for multiple pairs efficiently"""
        features = []
        for u, v, label in node_pairs:
            # Use caching for expensive computations
            cache_key = (u, v)
            if cache_key in self._cache:
                feat = self._cache[cache_key].copy()
                feat.update({'node_u': u, 'node_v': v, 'label': label})
            else:
                feat = self._extract_features_single(u, v, label)
                self._cache[cache_key] = {k: v for k, v in feat.items() 
                                         if k not in ['node_u', 'node_v', 'label']}
            features.append(feat)
        return pd.DataFrame(features)
    
    def _extract_features_single(self, u, v, label):
        """Extract features for a single pair"""
        preds = set(self.G.predecessors(u)) & set(self.G.predecessors(v))
        succs = set(self.G.successors(u)) & set(self.G.successors(v))
        feat = {
            'node_u': u, 'node_v': v, 'label': label,
            'common_predecessors': len(preds),
            'common_successors': len(succs),
            'out_in_degree_product': self.G.out_degree(u) * self.G.in_degree(v),
            'directional_adamic_adar': self._directed_adamic_adar(u, v, succs),
            'directional_resource_allocation': self._directed_resource_allocation(u, v, succs),
            'euclidian_distance': self._extract_euclid_dist(u, v)
        }
        feat.update(self._extract_capacity_features(u, v))
        return feat
    
    def _directed_adamic_adar(self, u, v, succs=None):
        """Optimized adamic adar calculation"""
        if succs is None:
            succs = set(self.G.successors(u)) & set(self.G.successors(v))
        score = 0
        for w in succs:
            deg = self.G.in_degree(w)
            if deg > 1:
                score += 1 / np.log(deg)
        return score
    
    def _directed_resource_allocation(self, u, v, succs=None):
        """Optimized resource allocation calculation"""
        if succs is None:
            succs = set(self.G.successors(u)) & set(self.G.successors(v))
        score = 0
        for w in succs:
            deg = self.G.in_degree(w)
            if deg > 0:
                score += 1 / deg
        return score
    
    def _extract_euclid_dist(self, u, v):
        """Fast Euclidean distance using precomputed positions"""
        x_u, y_u = self.positions[u]
        x_v, y_v = self.positions[v]
        return np.sqrt((x_u - x_v)**2 + (y_u - y_v)**2)
    
    def _extract_capacity_features(self, u, v):
        """Extract capacity features with caching"""
        def get_capacities(node, direction='out'):
            if direction == 'out':
                return [self.G[node][n].get('Overall_capacity', 0) for n in self.G.successors(node)]
            else:
                return [self.G[n][node].get('Overall_capacity', 0) for n in self.G.predecessors(node)]
        
        u_out = get_capacities(u, 'out')
        u_in = get_capacities(u, 'in')
        v_out = get_capacities(v, 'out')
        v_in = get_capacities(v, 'in')
        
        return {
            'avg_out_capacity_u': np.mean(u_out) if u_out else 0,
            'avg_in_capacity_u': np.mean(u_in) if u_in else 0,
            'avg_out_capacity_v': np.mean(v_out) if v_out else 0,
            'avg_in_capacity_v': np.mean(v_in) if v_in else 0,
            'min_out_capacity_u': min(u_out) if u_out else 0,
            'min_in_capacity_v': min(v_in) if v_in else 0
        }

# ------------------- PIPELINE -------------------
# 1️⃣ Mask edges for testing
G_train, masked_edges = mask_edges(G, mask_frac=0.2)
print(f"Masked {len(masked_edges)} edges out of {G.number_of_edges()}")

# 2️⃣ Precompute reachable nodes ONCE
reachable = compute_reachable_nodes(G_train, max_distance=2, cache_key='train')

# 3️⃣ Prepare datasets using precomputed reachable nodes
train_dataset = create_labeled_dataset_with_hard_negatives(
    G_train, cor, list(G_train.edges()), reachable_dict=reachable
)
test_dataset = create_labeled_dataset_with_hard_negatives(
    G_train, cor, masked_edges, reachable_dict=reachable
)

# 4️⃣ Extract features using optimized extractor
feature_extractor = FeatureExtractor(G_train, cor)
X_train = feature_extractor.extract_features_batch(train_dataset)
X_test = feature_extractor.extract_features_batch(test_dataset)

feature_cols = [c for c in X_train.columns if c not in ['node_u', 'node_v', 'label']]
y_train = X_train['label']
X_train = X_train[feature_cols]
y_test = X_test['label']
X_test = X_test[feature_cols]

# 5️⃣ Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 6️⃣ Evaluate on masked edges (true test set)
y_hat = rf_model.predict(X_test)
print("Performance on masked edges:")
print(classification_report(y_test, y_hat))

# Store the reachable dict for use in prediction
REACHABLE_FOR_PREDICTION = reachable

Masked 2228 edges out of 11140
Performance on masked edges:
              precision    recall  f1-score   support

           0       0.80      0.99      0.89      1513
           1       0.99      0.83      0.91      2228

    accuracy                           0.90      3741
   macro avg       0.90      0.91      0.90      3741
weighted avg       0.91      0.90      0.90      3741



5. Step
- assessment of the non-edge node pairs
- where should there be an existing edge, based on the patterns that random forest classifier learned?

In [ ]:
def predict_new_links_directed(G, trained_model, cor, feature_columns, 
                               max_distance=2, max_euclid_dist=0.02, 
                               reachable_dict=None):
    """
    Predict the probability that a directed edge should exist for candidate pairs.
    
    Parameters:
    - G: the graph (use full graph or training graph)
    - trained_model: your trained RF model
    - cor: DataFrame with node coordinates ['node','x','y']
    - feature_columns: columns used for training
    - max_distance: max BFS distance to generate candidate pairs
    - max_euclid_dist: max Euclidean distance for candidate pairs
    - reachable_dict: OPTIONAL precomputed reachable nodes (REQUIRED for efficiency!)
    """
    # Step 1: Generate candidate pairs
    positions = {row['node']: (row['x'], row['y']) for _, row in cor.iterrows()}
    candidate_pairs = []
    
    # Use precomputed reachable dict or compute new one
    if reachable_dict is not None:
        reachable = reachable_dict
    else:
        print("Warning: Computing reachable nodes from scratch - pass reachable_dict for efficiency")
        reachable = compute_reachable_nodes(G, max_distance, cache_key='predict')
    
    # Create set of existing edges for O(1) lookup
    existing_edges = set(G.edges())
    
    # Generate candidate pairs efficiently
    for u in G.nodes():
        x_u, y_u = positions[u]
        for v in reachable[u]:
            if (u, v) not in existing_edges:
                # Apply Euclidean distance limit
                x_v, y_v = positions[v]
                euclid_dist = np.sqrt((x_u - x_v)**2 + (y_u - y_v)**2)
                if euclid_dist <= max_euclid_dist:
                    candidate_pairs.append((u, v, -1))  # -1 label for unknown
    
    if not candidate_pairs:
        print("No candidate pairs found with given constraints")
        return pd.DataFrame(columns=['from_node', 'to_node', 'probability'])
    
    # Step 2: Extract features
    feature_extractor = FeatureExtractor(G, cor)
    X_candidates = feature_extractor.extract_features_batch(candidate_pairs)
    X_input = X_candidates[feature_columns]
    
    # Step 3: Predict probabilities
    probabilities = trained_model.predict_proba(X_input)[:, 1]
    
    # Step 4: Return DataFrame sorted by probability
    results = pd.DataFrame({
        'from_node': [u for u, v, _ in candidate_pairs],
        'to_node': [v for u, v, _ in candidate_pairs],
        'probability': probabilities
    })
    
    return results.sort_values('probability', ascending=False)

# ------------------- USAGE -------------------
# Use the reachable dict computed during training
best_new_links = predict_new_links_directed(
    G, rf_model, cor, feature_cols, 
    max_distance=2, 
    max_euclid_dist=0.005,
    reachable_dict=REACHABLE_FOR_PREDICTION  
)

print("Top 20 recommended new directed roads:")
print(best_new_links.head(20))

Top 20 recommended new directed roads:
      from_node  to_node  probability
4437       1291     2134         0.99
9319       4064     4060         0.99
8902       3955     3954         0.98
6701       4056     4057         0.98
1397       2338     1714         0.98
8057       2134     1291         0.98
9409       4414     4411         0.98
9406       4406     4413         0.97
9388       4360     4357         0.97
8701       2910     2876         0.97
6848       4212     4211         0.96
5780       4114     4110         0.94
4287       1250     2569         0.94
7167       3949     4134         0.94
7162       3921     4134         0.94
8674       2855     2856         0.93
9361       4164     2968         0.93
8741       3496     4365         0.93
8697       2879     4398         0.92
8687       2909     4396         0.92
